# Preprocessing transcripts

In [46]:
# Imports
import pickle
import pandas as pd
import numpy as np
import re

In [11]:
# Load transcripts from pickle
raw_transcripts = (
    pd.read_pickle('data/clean/subset_party_imputed_v2.p')
        .query('source == "parliament"')
        .drop(columns = ['index'])
    )  # TODO: change dataset after retrieval pipeline is cleaned up

In [12]:
print(raw_transcripts.shape)
raw_transcripts.head()

(92515, 6)


,doc,source,start_time,full_name,level,party
9,Tak. Danmarks kolonihistorie handler om militæ...,parliament,2019-10-03,Aki-Matilda Høegh-Dam,politician,SIU
10,"Jeg mener, at det er personen, der kan snakke ...",parliament,2019-10-03,Aki-Matilda Høegh-Dam,politician,SIU
11,"Det handler jo ikke om, hvad jeg synes. Det ha...",parliament,2019-10-03,Aki-Matilda Høegh-Dam,politician,SIU
12,"Jeg ved ikke, om det – når man snakker om at b...",parliament,2019-10-03,Aki-Matilda Høegh-Dam,politician,SIU
13,"Det er ikke noget, jeg bare mener; det er noge...",parliament,2019-10-03,Aki-Matilda Høegh-Dam,politician,SIU


In [58]:
query_string = 'fru'

[doc for doc in raw_transcripts['doc'] if query_string in doc][:25]

['Tak for ordet. I rigsfællesskabet går vi en spændende tid i møde, for vi er efter min mening på vej mod et reelt ligeværdigt fællesskab, og det er derfor meget vigtigt, at vi prioriterer tilliden mellem os. Derfor vil jeg også gerne som det første fra Siumuts side takke finansministeren og regeringen for at invitere os til at tage del i finanslovsforhandlingerne, da det ikke altid har været en selvfølge, at de nordatlantiske mandater blev indkaldt. Jeg ser forhåbningsfuldt frem til, at samarbejdet fortsætter på den måde i fremtiden. Desuden deltog vi også som observatører tidligt her i morges til diskussionerne om udmøntningen af reserven til foranstaltninger på social-, sundheds- og arbejdsmarkedsområdet. For selv om vi var observatører, blev vi altså behandlet ligeværdigt. Vi blev hørt, og vi blev lyttet til. Og det synes jeg altså også er et skridt i den rigtige retning, for det er noget, der genopbygger tilliden. Så lad os, så længe vi har den her konstellation med de to mandater

In [103]:
def remove_phrases(doc):
    """
    Remove phrases from a lowercased document.
    """
    
    # Remove names – requires uppercase and is thus done separately!
    name_pattern = r'(?:[Hh]r\. |[Ff]ru |[Ff]røken )[A-ZÆØÅ][a-zæøå]+(?:-[A-ZÆØÅ][a-zæøå]+)?(?:\s[A-ZÆØÅ][a-zæøå]+)*(?:-[A-ZÆØÅ][a-zæøå]+)?(?:\s[A-ZÆØÅ][a-zæøå]+)*(?:-[A-ZÆØÅ][a-zæøå]+)? '
    doc = re.sub(name_pattern, '', doc)

    # Consider removing party names and xxxministeren

    doc = doc.lower()

    # Remove procedural thank yous of different kinds
    politeness_pattern = r'^[\w\s,]*tak[\w\s,]*. '
    doc = re.sub(politeness_pattern, '', doc)
    
    return doc


In [104]:
test_doc = 'Tak. Fru Pernille Vermund brugte en del tid på at sværte fru Marie Krarup til, selv om fru Marie Krarup var så venlig at tilslutte sig beslutningsforslaget. Fru Pernille Vermunds anke var, at Dansk Folkeparti ikke havde gjort'

In [105]:
remove_phrases(test_doc)

'brugte en del tid på at sværte til, selv om var så venlig at tilslutte sig beslutningsforslaget. anke var, at dansk folkeparti ikke havde gjort'